In [2]:
import pandas as pd
order_products_smol = pd.read_csv("../instacart_2017_05_01/order_products__train.csv")
aisles = pd.read_csv("../instacart_2017_05_01/aisles.csv")
departments = pd.read_csv("../instacart_2017_05_01/departments.csv")
products = pd.read_csv("../instacart_2017_05_01/products.csv")
orders = pd.read_csv("../instacart_2017_05_01/orders.csv")

In [3]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [4]:
user_orders = orders.merge(order_products_smol,on='order_id', how='inner')

In [5]:
user_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,1187899,1,train,11,4,8,14.0,196,1,1
1,1187899,1,train,11,4,8,14.0,25133,2,1
2,1187899,1,train,11,4,8,14.0,38928,3,1
3,1187899,1,train,11,4,8,14.0,26405,4,1
4,1187899,1,train,11,4,8,14.0,39657,5,1


In [6]:
user_orders = user_orders.merge(products, on='product_id', how='inner')

In [7]:
user_orders = user_orders.merge(aisles, on='aisle_id', how='inner')
user_orders = user_orders.merge(departments, on='department_id', how='inner')
user_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,1187899,1,train,11,4,8,14.0,196,1,1,Soda,77,7,soft drinks,beverages
1,2757217,67,train,25,0,11,5.0,196,1,1,Soda,77,7,soft drinks,beverages
2,632715,676,train,12,0,13,26.0,196,2,1,Soda,77,7,soft drinks,beverages
3,1167274,760,train,5,4,10,8.0,196,2,1,Soda,77,7,soft drinks,beverages
4,3347074,804,train,16,3,21,5.0,196,4,0,Soda,77,7,soft drinks,beverages


In [8]:
products_per_user = user_orders[['user_id', 'product_id']]
products_per_user.head()

,user_id,product_id
0,1,196
1,67,196
2,676,196
3,760,196
4,804,196


In [9]:
products_per_user.sort_values('user_id')[:20]

,user_id,product_id
0,1,196
378536,1,10258
367083,1,39657
331097,1,26405
485945,1,13032
243629,1,49235
210985,1,27845
155745,1,38928
391068,1,26088
1507,1,46149


In [10]:
products_per_user = products_per_user.groupby('user_id', as_index=False).agg(list)

In [11]:
products_per_user.head()

,user_id,product_id
0,1,"[196, 46149, 25133, 38928, 27845, 49235, 26405..."
1,2,"[24838, 11913, 32792, 16589, 38547, 45645, 229..."
2,5,"[16185, 20843, 19057, 20114, 40706, 21616, 153..."
3,7,"[12053, 47272, 37999, 43967, 17638, 40852, 450..."
4,8,"[25659, 5539, 47766, 15937, 23165, 4853, 27104..."


In [12]:
def string_list(product_id):
    return [str(i) for i in product_id]

In [13]:
products_per_user['product_id'] = products_per_user['product_id'].apply(string_list)
products_per_user.head()

,user_id,product_id
0,1,"[196, 46149, 25133, 38928, 27845, 49235, 26405..."
1,2,"[24838, 11913, 32792, 16589, 38547, 45645, 229..."
2,5,"[16185, 20843, 19057, 20114, 40706, 21616, 153..."
3,7,"[12053, 47272, 37999, 43967, 17638, 40852, 450..."
4,8,"[25659, 5539, 47766, 15937, 23165, 4853, 27104..."


In [14]:
def join_products(product_id_str):
    return ' '.join(product_id_str)

In [15]:
products_per_user['product_id'] = products_per_user['product_id'].apply(join_products)
products_per_user.head()

,user_id,product_id
0,1,196 46149 25133 38928 27845 49235 26405 39657 ...
1,2,24838 11913 32792 16589 38547 45645 22963 1901...
2,5,16185 20843 19057 20114 40706 21616 15349 2141...
3,7,12053 47272 37999 43967 17638 40852 45066 2989...
4,8,25659 5539 47766 15937 23165 4853 27104 7058 4...


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(products_per_user['product_id'])

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)